In [1]:
%%time
import pandas as pd

from sklearn.metrics import roc_auc_score, make_scorer, accuracy_score
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.preprocessing import StandardScaler, Normalizer, MinMaxScaler


from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score

from xgboost import XGBClassifier
from sklearn.feature_selection import RFE


from time import strftime, strptime, mktime, time, ctime

from sklearn import model_selection

from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB 
from sklearn.svm import LinearSVC

from xgboost import XGBClassifier
import xgboost as xgb

from sklearn.tree import DecisionTreeClassifier, ExtraTreeClassifier
from sklearn.ensemble import AdaBoostClassifier, BaggingClassifier,\
ExtraTreesClassifier, GradientBoostingClassifier, VotingClassifier, RandomForestClassifier

from mlxtend.classifier import StackingClassifier, StackingCVClassifier
from mlxtend.feature_selection import ColumnSelector

from sklearn.pipeline import make_pipeline
import numpy as np

import gc



/home/shj16110/anaconda2/lib/python2.7/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


CPU times: user 591 ms, sys: 2.21 s, total: 2.8 s
Wall time: 4.1 s


In [2]:
%%time
df = pd.read_csv('./train.csv')
print df.shape
test = pd.read_csv('./test.csv')
print test.shape

y = df.pop('Approved')
train_id = df.pop('ID')
test_id = test.pop('ID')

def helper(x):
    try:
        tmp = strptime(x, '%d/%m/%y')
        tmp = mktime(tmp)
    except:
        tmp = 563000400
    return tmp

df['Lead_Creation_Date'] = df['Lead_Creation_Date'].apply(helper)
test['Lead_Creation_Date'] = test['Lead_Creation_Date'].apply(helper)

df['DOB'] = df['DOB'].apply(helper)
test['DOB'] = test['DOB'].apply(helper)

df = df.fillna(axis=1, value=-1)
test = test.fillna(axis=1, value=-1)

df = pd.get_dummies(df)
test = pd.get_dummies(test)
test = test.reindex(columns = df.columns, fill_value=0)

test_o = test.copy()

(69713, 22)
(30037, 21)
CPU times: user 13.2 s, sys: 7.01 s, total: 20.2 s
Wall time: 20.3 s


In [29]:
%%time

rid = []

for i in df:
    if df[i].value_counts().head(1).values[0]>df.shape[0]*0.999:
        rid.append(i)
print len(rid), df.shape[1]-len(rid)

37230 186
CPU times: user 33.8 s, sys: 1e+03 µs, total: 33.8 s
Wall time: 33.8 s


### from 186 features

In [104]:
from keep import keep
record = []

In [80]:
dic = {}
for i in keep:
    dic[i]=df[i].value_counts().head(1).values[0]


In [84]:
pic = pd.Series(dic).sort_values(ascending=True)
rm = list(pic.index)

In [85]:
len(rm)

186

In [114]:
RANDOM_STATE = 2018
params = {
    'min_child_weight': 0.8,
#     'eta': 0.1,
    'colsample_bytree': 0.8,
    'max_depth': 12,
    'subsample': 0.8,
    'alpha': 0.3,
    'lambda':0.9,
    'gamma': 0.3,
    'silent': 0,
    'learning_rate':0.02,
    'verbose_eval': True,
    'seed': RANDOM_STATE,
    'objective':"binary:logistic",
#     'eval_metric':'error'
#     'num_class':1
    'n_jobs':24,
    'eval_metric':'auc'
    
}



In [108]:
df.shape

(69713, 37416)

In [115]:
def search(i):
    global record
    
    update_rid = set(df.columns) - set(rm[:i])
    print len(df.columns) - len(update_rid)


    train = df.drop(update_rid, axis=1)
    test = test_o.drop(update_rid, axis=1)

    s = StandardScaler()
    
    train = s.fit_transform(train)
    test = s.transform(test)

    print train.shape, test.shape    
    
    m = lambda *x: [i.shape for i in x]
    X_train, X_test, y_train, y_test = train_test_split(train, y, stratify=y, test_size=0.2, random_state=23333)
    print m(X_train, y_train, X_test, y_test)

    d_train = xgb.DMatrix(X_train, label=y_train)
    d_test = xgb.DMatrix(X_test, label=y_test)    
    
    watchlist = [(d_train, 'train'), (d_test, 'test')]

    
    
    start=time()

    clf = xgb.train(params, d_train, 10000, watchlist, early_stopping_rounds=200, 
                    verbose_eval=200,  maximize=True,
                   )



    print '!!! used %.2f s'%(time()-start)    
    record.append([i, clf.best_iteration, clf.best_score])
    print record[-1]

In [93]:
clf.best_iteration

272

In [116]:
for i in range(50, 151):
    search(i)
len(record)

50
(69713, 50) (30037, 50)
[(55770, 50), (55770,), (13943, 50), (13943,)]
[0]	train-auc:0.737913	test-auc:0.73824
Multiple eval metrics have been passed: 'test-auc' will be used for early stopping.

Will train until test-auc hasn't improved in 200 rounds.
[200]	train-auc:0.975102	test-auc:0.852563
[400]	train-auc:0.996802	test-auc:0.853506
Stopping. Best iteration:
[262]	train-auc:0.987022	test-auc:0.856721

!!! used 9.56 s
[50, 262, 0.856721]
51
(69713, 51) (30037, 51)
[(55770, 51), (55770,), (13943, 51), (13943,)]
[0]	train-auc:0.744569	test-auc:0.751996
Multiple eval metrics have been passed: 'test-auc' will be used for early stopping.

Will train until test-auc hasn't improved in 200 rounds.
[200]	train-auc:0.97505	test-auc:0.854532
[400]	train-auc:0.996585	test-auc:0.854206
Stopping. Best iteration:
[257]	train-auc:0.986575	test-auc:0.858229

!!! used 14.73 s
[51, 257, 0.858229]
52
(69713, 52) (30037, 52)
[(55770, 52), (55770,), (13943, 52), (13943,)]
[0]	train-auc:0.744569	test-a

Multiple eval metrics have been passed: 'test-auc' will be used for early stopping.

Will train until test-auc hasn't improved in 200 rounds.
[200]	train-auc:0.975686	test-auc:0.854965
[400]	train-auc:0.99679	test-auc:0.854497
Stopping. Best iteration:
[269]	train-auc:0.988085	test-auc:0.858081

!!! used 11.23 s
[68, 269, 0.858081]
69
(69713, 69) (30037, 69)
[(55770, 69), (55770,), (13943, 69), (13943,)]
[0]	train-auc:0.738377	test-auc:0.746472
Multiple eval metrics have been passed: 'test-auc' will be used for early stopping.

Will train until test-auc hasn't improved in 200 rounds.
[200]	train-auc:0.97611	test-auc:0.853426
[400]	train-auc:0.996765	test-auc:0.855996
Stopping. Best iteration:
[333]	train-auc:0.993692	test-auc:0.857736

!!! used 12.90 s
[69, 333, 0.857736]
70
(69713, 70) (30037, 70)
[(55770, 70), (55770,), (13943, 70), (13943,)]
[0]	train-auc:0.744311	test-auc:0.752663
Multiple eval metrics have been passed: 'test-auc' will be used for early stopping.

Will train until 

[200]	train-auc:0.973193	test-auc:0.853447
[400]	train-auc:0.996195	test-auc:0.855025
Stopping. Best iteration:
[304]	train-auc:0.990837	test-auc:0.857369

!!! used 19.70 s
[86, 304, 0.857369]
87
(69713, 87) (30037, 87)
[(55770, 87), (55770,), (13943, 87), (13943,)]
[0]	train-auc:0.74433	test-auc:0.752658
Multiple eval metrics have been passed: 'test-auc' will be used for early stopping.

Will train until test-auc hasn't improved in 200 rounds.
[200]	train-auc:0.974377	test-auc:0.852683
[400]	train-auc:0.996255	test-auc:0.85584
Stopping. Best iteration:
[307]	train-auc:0.991405	test-auc:0.857117

!!! used 20.85 s
[87, 307, 0.857117]
88
(69713, 88) (30037, 88)
[(55770, 88), (55770,), (13943, 88), (13943,)]
[0]	train-auc:0.746157	test-auc:0.752416
Multiple eval metrics have been passed: 'test-auc' will be used for early stopping.

Will train until test-auc hasn't improved in 200 rounds.
[200]	train-auc:0.975117	test-auc:0.854065
[400]	train-auc:0.996393	test-auc:0.858138
Stopping. Best i

Stopping. Best iteration:
[250]	train-auc:0.984129	test-auc:0.859012

!!! used 14.27 s
[104, 250, 0.859012]
105
(69713, 105) (30037, 105)
[(55770, 105), (55770,), (13943, 105), (13943,)]
[0]	train-auc:0.741035	test-auc:0.741243
Multiple eval metrics have been passed: 'test-auc' will be used for early stopping.

Will train until test-auc hasn't improved in 200 rounds.
[200]	train-auc:0.974982	test-auc:0.854102
[400]	train-auc:0.996543	test-auc:0.855601
Stopping. Best iteration:
[300]	train-auc:0.991049	test-auc:0.857402

!!! used 23.74 s
[105, 300, 0.857402]
106
(69713, 106) (30037, 106)
[(55770, 106), (55770,), (13943, 106), (13943,)]
[0]	train-auc:0.741035	test-auc:0.741243
Multiple eval metrics have been passed: 'test-auc' will be used for early stopping.

Will train until test-auc hasn't improved in 200 rounds.
[200]	train-auc:0.974459	test-auc:0.855118
[400]	train-auc:0.996009	test-auc:0.856934
Stopping. Best iteration:
[297]	train-auc:0.990067	test-auc:0.860536

!!! used 21.59 s
[

!!! used 19.70 s
[122, 260, 0.857316]
123
(69713, 123) (30037, 123)
[(55770, 123), (55770,), (13943, 123), (13943,)]
[0]	train-auc:0.740091	test-auc:0.741527
Multiple eval metrics have been passed: 'test-auc' will be used for early stopping.

Will train until test-auc hasn't improved in 200 rounds.
[200]	train-auc:0.974557	test-auc:0.853416
[400]	train-auc:0.996283	test-auc:0.856485
Stopping. Best iteration:
[326]	train-auc:0.992675	test-auc:0.857753

!!! used 28.19 s
[123, 326, 0.857753]
124
(69713, 124) (30037, 124)
[(55770, 124), (55770,), (13943, 124), (13943,)]
[0]	train-auc:0.74433	test-auc:0.752664
Multiple eval metrics have been passed: 'test-auc' will be used for early stopping.

Will train until test-auc hasn't improved in 200 rounds.
[200]	train-auc:0.973858	test-auc:0.855191
[400]	train-auc:0.995975	test-auc:0.854278
Stopping. Best iteration:
[227]	train-auc:0.979932	test-auc:0.857792

!!! used 20.75 s
[124, 227, 0.857792]
125
(69713, 125) (30037, 125)
[(55770, 125), (55770

Exception AttributeError: "'Booster' object has no attribute 'handle'" in <bound method Booster.__del__ of <xgboost.core.Booster object at 0x2b9be7cd8050>> ignored


KeyboardInterrupt: 

In [129]:
rs = pd.DataFrame(record, columns=['feature','round','score'])
# rs

In [120]:
rs.to_csv('search1.csv',index=False)

In [127]:
rs[rs['score']>.859].sort_values(by=['score'],ascending=False)

,feature,round,score
78,128,243,0.862080
5,55,293,0.861089
35,85,301,0.860633
56,106,297,0.860536
60,110,318,0.860304
79,129,331,0.860284
71,121,357,0.860187
38,88,302,0.860144
65,115,310,0.860101
7,57,269,0.859971


In [186]:
update_rid = set(df.columns) - set(rm[:115])
print len(df.columns) - len(update_rid)


train = df.drop(update_rid, axis=1)
test = test_o.drop(update_rid, axis=1)

s = StandardScaler()

train = s.fit_transform(train)
test = s.transform(test)

print train.shape, test.shape    

m = lambda *x: [i.shape for i in x]
X_train, X_test, y_train, y_test = train_test_split(train, y, stratify=y, test_size=0.2, random_state=23333)
print m(X_train, y_train, X_test, y_test)

d_train = xgb.DMatrix(X_train, label=y_train)
d_test = xgb.DMatrix(X_test, label=y_test)    

watchlist = [(d_train, 'train'), (d_test, 'test')]


115
(69713, 115) (30037, 115)
[(55770, 115), (55770,), (13943, 115), (13943,)]


### from here

In [187]:
RANDOM_STATE = 2018
params = {
    'min_child_weight': 0.8,
#     'eta': 0.1,
    'colsample_bytree': 0.8,
    'max_depth': 12,
    'subsample': 0.8,
    'alpha': 0.3,
    'lambda':0.9,
    'gamma': 0.3,
    'silent': 0,
    'learning_rate':0.02,
    'verbose_eval': True,
    'seed': RANDOM_STATE,
    'objective':"binary:logistic",
    'n_jobs':24,
    'eval_metric':'auc'   
}

In [188]:
start=time()

clf = xgb.train(params, d_train, 310, watchlist, early_stopping_rounds=200, 
                verbose_eval=50,  maximize=True,
               )
                


'!!! used %.2f s'%(time()-start)

[0]	train-auc:0.740091	test-auc:0.741527
Multiple eval metrics have been passed: 'test-auc' will be used for early stopping.

Will train until test-auc hasn't improved in 200 rounds.
[50]	train-auc:0.867942	test-auc:0.837898
[100]	train-auc:0.915882	test-auc:0.848035
[150]	train-auc:0.952024	test-auc:0.853689
[200]	train-auc:0.973697	test-auc:0.85658
[250]	train-auc:0.984156	test-auc:0.859146
[300]	train-auc:0.990248	test-auc:0.859504


'!!! used 10.18 s'

In [189]:
clf.best_iteration, clf.best_score

(309, 0.860057)

In [190]:
%%time
dtest = xgb.DMatrix(test)
predb = clf.predict(dtest)

submit = pd.concat([pd.DataFrame(test_id.values, columns=['ID']), pd.DataFrame(predb, columns=['Approved'])], axis=1)
submit.shape

CPU times: user 1.1 s, sys: 0 ns, total: 1.1 s
Wall time: 91.7 ms


___

In [191]:
submit.to_csv('./submit/xgb_115.csv',index=False)

#### lb .852388

In [60]:
# clf.save_model('xgb_0.852586.model')

In [441]:
roc_auc_score?

In [442]:
submit2 = pd.concat([pd.DataFrame(test_id.values, columns=['ID']), pd.DataFrame(pred, columns=['Approved'])], axis=1)
submit2.shape

(30037, 2)

In [449]:
accuracy_score(y_test, [0]*y_test.shape[0])

0.9834325467976762

___

In [276]:
cross_val_score(RandomForestClassifier(n_estimators=50, n_jobs=24), df.iloc[:,:-1], df.iloc[:,-1], cv=5)

array([0.87807502, 1.        , 0.99992828, 1.        , 0.98759145])

In [76]:
fr = RandomForestClassifier(n_estimators=50)
fr

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=50, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [77]:
fr.fit(df.iloc[:,:-1], df.iloc[:,-1])

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=50, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [ ]:
fr.predict_log_proba